# Tensorflow 기본 사용법
* tf.constant
* tf.Session
* tf.Variable
* tf.placeholder
* tf.graph
* life cycle

In [ ]:
import tensorflow as tf

## 상수
* tf.constant()로 생성하는 상수는 초기화 없이 사용이 가능하다. 
    * 하지만 값의 변경이 되지 않는다
* 값은 세션에 저장된다.


In [ ]:
hello = tf.constant('Hello World!')
print(hello)
#dir(hello)

In [ ]:
a = tf.constant(10)
b = tf.constant(20)
c = a + b
print(c)

In [ ]:
cc = tf.add(a, b)
print(cc)

## Session
### `tf.Session()`
* 노드들로 구성된 그래프를 실행하려면 Session이 필요하다.
* 그래프를 CPU 또는 GPU에 로드하고 실행한다.
    * `run()`
* `sess = tf.Session()`으로 생성할 수 있고 `sess.run()`으로 텐서를 실행할 수 있다.
* 모든 tensor들은 `t.eval()` 메서드를 가지고 있는데, 이것은 `tf.get_default_session.run(t)` 한것과 같다.
    * 또는 `t.eval(session=sess)`로 세션을 지정할 수 도 있다.
* `with tf.Session() as sess :` 구문으로 블록내의 default session을 지정할 수 있다.
* `with sess.as_default()` 함수로 default session을 지정할 수도 있다.
* `sess.close()` 함수로 자원을 반납해야 한다.

### `tf.InteractiveSession()`
* 자동으로 전역으로 사용할 default session으로 등록하게 된다.
* with 구문이 필요없다.

In [ ]:
sess = tf.Session()
#dir(sess)

In [ ]:
sess.run(c)

In [ ]:
sess.run(cc)

In [ ]:
sess.run(a)

## 변수 , `tf.Variable()`
* 값을 변경할 수 있는 변수
* `tf.Variable(val)`
* 보통 학습 모델에 의해 Wegith 값을 보관할 목적으로 사용
    * 학습 과정에서 값이 변경된다.
    * 초기 값을 주기 애매 할 때는 랜덤 값등을 사용 할 수 있다.
    * `tf.Variable(tf.random_normal([n, m])`
* 사용하기 전에 초기화를 반드시 수행해야 한다.
    *  sess.run(t.initializer) 
    * t.initializer.run()
        * default session 이 있는 경우
* 여러 변수를 한꺼번에 초기화 할때
    * `sess.run(tf.global_variables_initializer())`
    * `tf.global_variable_initialzer.run()` 
        * default session 있는 경우

In [ ]:
x = tf.Variable(3)
y = tf.Variable(4)
z = x * y
print(x, y, z)

In [ ]:
print(tf.get_default_session())

In [ ]:
print(tf.global_variables_initializer())

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
sess.run([x,y,z])

In [ ]:
z.eval(session=sess)

In [ ]:
with sess.as_default():
    print(z.eval())

In [ ]:
sess2 = tf.InteractiveSession()

In [ ]:
x.initializer.run()
y.initializer.run()
z.eval()

## PlaceHolder
* 매개변수, 식은 정해져 있지만 값은 실행할때 마다 달라져야 하는 것들
* `tf.placeholder(tf.float32, [None, 2])`
* 실제로 값을 전달하는 곳은 `sess.run()`에서 `feed_dict={A:a, B:b}` 형식으로 전달

In [ ]:
tf.reset_default_graph()
x_data = [1,2,3,4,5]
X = tf.placeholder(tf.int32)
adder = tf.constant(10)
y = tf.Variable(0)
y = X + adder
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(y, feed_dict={X:x_data}))


### 그래프, graph
* 상수, 변수 등의 텐서를 생성하면 이것은 하나의 계산 그래프에 추가된다.
* 각 텐서의 `t.graph` 속성에 지정된다
* 새로운 그래프를 만드려면 `tf.Graph()` 함수가 있다.
* 새로운 그래프를 사용하려면 `with graph.as_default():`로 블럭 설정을 해서 사용할 수 있다.
* 텐서를 생성하는것과 같은 코드를 반복해서 실행하면 그래프에 중복된 텐서가 텐서 노드가 포함되기 때문에 오류가 발생
    * 파이썬 프로세스 또는 커널을 재 실행 또는
    * `tf.reset_default_graph()`를 실행
 

In [ ]:
#??

## 노드의 Life Cycle
* 종속성 있는 노드는 평가 시점에 모두 평가가 같이 실행된다.
* 종속성있는 노드를 2번 평가되면 그 과정의 노드들도 2번 평가된다.
* 한번에 하고 싶으면 리스트로 만들어서 한번에 평가하라.

### 세션을 생성하고 닫은 후 다시 실행하면 값을 항상 처음으로 돌아 간다.

In [ ]:
w = tf.constant(2)
x = tf.Variable(3)
y =  w + x

sess = tf.Session()
sess.run(tf.global_variables_initializer())

y = y *2 
z = y + 1

print(sess.run(w), sess.run(x), sess.run(y))
print(sess.run(z))

sess.close()

### 하지만 세션을 닫지 않고 실행만 다시 하면 값을 유지된다.

In [ ]:
w = tf.constant(2)
x = tf.Variable(3)
y =  w + x

sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [ ]:
y = y *2 
z = y + 1

print(sess.run(w), sess.run(x), sess.run(y))
print(sess.run(z))


### 세션을 실행할때 개별적으로 실행하면 종속성있는 모든 노드가 매번 실행 된다.
* 아래의 경우 z의 값이 y의 값을 2번 변경하게 한다.

In [ ]:
w = tf.constant(2)
x = tf.Variable(3)
y = tf.Variable(0)
y =  tf.assign(y, y + x)
z = y + 1
sess = tf.Session()
sess.run(tf.global_variables_initializer())


print(sess.run(w), sess.run(x), sess.run(y), sess.run(z))
sess.close()

### 세션에 노드를 리스트로 묶어서 실행하면 한번에 평가된다.
* 아래의 코드의 z는 y를 두번 평가하게 하지 않는다.

In [ ]:
w = tf.constant(2)
x = tf.Variable(3)
y = tf.Variable(0)
y =  tf.assign(y, y + x)
z = y + 1
sess = tf.Session()
sess.run(tf.global_variables_initializer())


print(sess.run([w,x,y,z]))

sess.close()